# 수어 인식 모델 - CNN + LSTM + Attention

이 노트북은 수어 인식을 위한 CNN + LSTM 하이브리드 모델을 구현합니다.

## 주요 특징
- 버킷 기반 시퀀스 길이 정규화
- 에러 처리 및 검증
- GPU 최적화
- 실시간 예측 지원


## 1. 라이브러리 및 초기 설정

- pip은 GPU 서버에서 돌리기에는 충돌(tensor 충돌 때문에 GPU를 탐지하지를 못함)때문에 사용하지 않음. 로컬에서는 괜찮음
- 재현성을 위해 np tf의 시드를 42로 고정

In [ ]:
# %pip install -r requirements.txt

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from pathlib import Path
import pickle
import warnings
warnings.filterwarnings('ignore')

from config import ModelConfig as cf

print(f"TensorFlow 버전: {tf.__version__}")
print(f"GPU 사용 가능: {tf.config.list_physical_devices('GPU')}")

np.random.seed(42)
tf.random.set_seed(42)


TensorFlow 버전: 2.14.0
GPU 사용 가능: []


## 2. 시퀀스 길이 정규화 클래스


In [3]:
class NormalizeSequenceLength:
    """
    시퀀스 길이를 정규화하는 클래스
    - 패딩: 짧은 시퀀스를 0으로 채움
    - 트리밍: 긴 시퀀스를 균등 샘플링
    """
    
    def pad_or_trim(self, seq: np.ndarray, target_len: int) -> np.ndarray:
        """
        시퀀스를 target_len 길이로 정규화
        
        Args:
            seq: 입력 시퀀스 (2D array)
            target_len: 목표 길이
            
        Returns:
            정규화된 시퀀스
        """
        try:
            # 입력 검증
            if seq is None or seq.size == 0:
                raise ValueError("입력 시퀀스가 비어있습니다.")
            
            if len(seq.shape) != 2:
                raise ValueError(f"시퀀스는 2차원 배열이어야 합니다. 현재 shape: {seq.shape}")
            
            t, F = seq.shape
            
            if target_len <= 0:
                raise ValueError(f"target_len은 양수여야 합니다. 현재 값: {target_len}")
            
            # 길이가 같으면 그대로 반환
            if t == target_len:
                return seq
            
            # 길이가 길면 샘플링
            if t > target_len:
                idx = np.linspace(0, t - 1, target_len).astype(int)
                return seq[idx]
            
            # 길이가 짧으면 패딩
            pad = np.zeros((target_len - t, F), dtype=seq.dtype)
            return np.vstack([seq, pad])
            
        except Exception as e:
            print(f"pad_or_trim 에러: {e}")
            raise

    def nearest_bucket_len(self, t: int, buckets=cf.BUCKETS) -> int:
        """
        현재 길이와 가장 가까운 버킷 길이를 반환
        
        Args:
            t: 현재 시퀀스 길이
            buckets: 사용 가능한 버킷 길이들
            
        Returns:
            가장 가까운 버킷 길이
        """
        try:
            if t <= 0:
                raise ValueError(f"시퀀스 길이는 양수여야 합니다. 현재 값: {t}")
            
            if not buckets or len(buckets) == 0:
                raise ValueError("버킷 리스트가 비어있습니다.")
            
            if any(b <= 0 for b in buckets):
                raise ValueError("모든 버킷 값은 양수여야 합니다.")
            
            return min(buckets, key=lambda b: abs(b - t))
            
        except Exception as e:
            print(f"nearest_bucket_len 에러: {e}")
            raise

# 테스트
normalizer = NormalizeSequenceLength()
test_seq = np.random.rand(15, 225)  # 15프레임 시퀀스
normalized = normalizer.pad_or_trim(test_seq, 60)
print(f"원본: {test_seq.shape} → 정규화: {normalized.shape}")


원본: (15, 225) → 정규화: (60, 225)


## 3. 수어 인식 모델 클래스


In [4]:
class SignLanguageModel:
    """
    수어 인식을 위한 CNN + LSTM 하이브리드 모델
    
    특징:
    - TimeDistributed CNN으로 프레임별 특징 추출
    - LSTM으로 시계열 패턴 학습
    - 버킷 기반 시퀀스 길이 정규화
    """
    
    def __init__(self):
        """모델 초기화"""
        self.num_classes = cf.NUM_CLASSES
        self.sequence_length = cf.SEQUENCE_LENGTH
        self.feature_dim = cf.FEATURE_DIM
        self.model = None
        self.label_encoder = LabelEncoder()
        self.normalizer = NormalizeSequenceLength()
        
        print(f"모델 설정:")
        print(f"  - 시퀀스 길이: {self.sequence_length}")
        print(f"  - 특성 차원: {self.feature_dim}")
        print(f"  - 클래스 수: {self.num_classes}")
        print(f"  - 버킷: {cf.BUCKETS}")


### 3.1 모델 구성
- CNN 3계층
- BiLSTM
- Attention
- Flatten


In [ ]:
def build_model(self):
    """
    CNN + LSTM 하이브리드 모델 구성
    
    구조:
    1. TimeDistributed CNN (프레임별 특징 추출)
    2. LSTM (시계열 패턴 학습)
    3. Dense (분류)
    """
    try:
        # 입력 검증
        if self.sequence_length <= 0 or self.feature_dim <= 0:
            raise ValueError(f"잘못된 입력 크기: sequence_length={self.sequence_length}, feature_dim={self.feature_dim}")
        
        print("모델 구성 중...")
        
        # 입력층
        inputs = keras.Input(shape=(self.sequence_length, self.feature_dim))
        
        # CNN을 위한 reshape    
        # Conv1D(1차원 합성곱)은 입력을 길이(sequence_length), 채널(feature_dim)으로 받음
        # 아래의 TimeDistributed를 사용하려면 프레임마다 (F,1)로 보이게 해야되므로 reshape 하는 것
        x = layers.Reshape((self.sequence_length, self.feature_dim, 1))(inputs)
        
        # TimeDistributed CNN 블록 1
        # 프레임 별로 Conv1D를 적용, 3은 인접 피처 3개를 한번에 포며 로컬 패턴을 잡는 다는 것
        # BN으로 분포 안정화, MaxPooling으로 피처 길이를 반으로 잘라 요약, Dropout으로 과적합 완화
        x = layers.TimeDistributed(layers.Conv1D(64, 3, activation='relu', padding='same'))(x)
        x = layers.TimeDistributed(layers.BatchNormalization())(x)
        x = layers.TimeDistributed(layers.MaxPooling1D(2))(x)
        x = layers.TimeDistributed(layers.Dropout(cf.DROPOUT_RATE))(x)
        
        # TimeDistributed CNN 블록 2
        # 채널(128)을 늘려 풍부한 패턴으로 표현, Pool로 다시 절반으로 자름
        x = layers.TimeDistributed(layers.Conv1D(128, 3, activation='relu', padding='same'))(x)
        x = layers.TimeDistributed(layers.BatchNormalization())(x)
        x = layers.TimeDistributed(layers.MaxPooling1D(2))(x)
        x = layers.TimeDistributed(layers.Dropout(cf.DROPOUT_RATE))(x)
        
        # TimeDistributed CNN 블록 3
        # 256채널까지로 늘려 표현, GlobalMaxPooling1D를 사용해 프레임 내부 길이축(F/4)를 완전히 접어
        # 프레임을 256차원 벡터로 축약
        # 텐서는 시퀀스 * 프레임 벡터 형태가 되어 LSTM을 위한 입력이 됨.
        x = layers.TimeDistributed(layers.Conv1D(256, 3, activation='relu', padding='same'))(x)
        x = layers.TimeDistributed(layers.BatchNormalization())(x)
        x = layers.TimeDistributed(layers.GlobalMaxPooling1D())(x)
        x = layers.TimeDistributed(layers.Dropout(cf.DROPOUT_RATE))(x)
        
        # LSTM 블록
        # 512와 256은 리턴 시퀀스가 T라는 것은 모든 타임스텝의 출력을 다음 LSTM으로 전달하겠다는 뜻.
        # 그래서 128은 마지막 층이기에 리턴시퀀스가 없음, 대신 요약 벡터만 남겨 문맥 응축
        x = layers.Bidirectional(layers.LSTM(512, return_sequences=True, dropout=cf.DROPOUT_RATE))(x)
        x = layers.Bidirectional(layers.LSTM(256, return_sequences=True, dropout=cf.DROPOUT_RATE))(x)
        x = layers.Bidirectional(layers.LSTM(128, return_sequences=True, dropout=cf.DROPOUT_RATE))(x)
        
        # Attention 블록.
        # BiLSTM까지 끝난 데이터를 Attention에 먹여서 가중치를 학습
        attention = layers.Attention()([x, x])
        x = layers.Add()([x, attention])
        x = layers.GlobalAveragePooling1D()(x)
        
        # faltten 으로 시퀀스 전체 백터화
        # GlobalAveragePooling1D 때문에 2차원으로 변환되므로 Flatten이 의미를 잃음.
        #x = layers.Flatten()(x)
        x = layers.Dropout(0.3)(x)
        
        # 분류기
        # LSTM의 123차원을 비선형 변환으로 확장(512->256), BN/Dropout으로 일반화 성능 확보
        x = layers.Dense(512, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.5)(x)
        
        x = layers.Dense(256, activation='relu')(x)
        # 나오는 학습률 결과에 따라 0.3 / 0.4 수준으로 한번 더 넣을지 결정
        # 혹은 아예 LSTM 수준에서 조정을 하고 0.5 두번을 넣는 방안도 결과 보고 결정하자.
        # x = layers.Dropout(0.5)(x) 
        
        # 로짓을 클래스 수 만큼 만들어 softmax 확률을 출력
        outputs = layers.Dense(self.num_classes, activation='softmax')(x)
        

        # 모델 생성
        self.model = keras.Model(inputs, outputs)
        
        # 컴파일
        #clipnorm=1.0 == 그레디언트 폭주 방지(LSTM에서 사용하려면 꼭 해줘야 한다고 함)
        # accuracy = 정답 정확도
        # top_5_accuracy = 정답이 상위 5개 예측(확률) 안에 들어갔는지
        loss = keras.losses.CategoricalCrossentropy(label_smoothing=0.1)
        self.model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=cf.LEARNING_RATE, clipnorm=1.0),
            loss=loss,
            metrics=['accuracy', 'top_5_accuracy']
        )
        
        print("모델 구성 완료!")
        return self.model
        
    except Exception as e:
        print(f"build_model 에러: {e}")
        raise

# SignLanguageModel 클래스에 메서드 추가
SignLanguageModel.build_model = build_model

### 3.2 시퀀스 길이 정규화


In [6]:
def normalize_sequence_length(self, sequence: np.ndarray) -> np.ndarray:
    """
    시퀀스 길이를 고정 길이로 정규화
    
    Args:
        sequence: 입력 시퀀스 (2D array)
        
    Returns:
        정규화된 시퀀스 (SEQUENCE_LENGTH 길이)
    """
    try:
        # 입력 검증
        if sequence is None or sequence.size == 0:
            raise ValueError("입력 시퀀스가 비어있습니다.")
        
        if len(sequence.shape) != 2:
            raise ValueError(f"시퀀스는 2차원 배열이어야 합니다. 현재 shape: {sequence.shape}")
        
        # NaN 값 처리
        sequence = np.nan_to_num(sequence, copy=False).astype(np.float32)
        
        # 고정 길이로 정규화
        seq = self.normalizer.pad_or_trim(sequence, cf.SEQUENCE_LENGTH)
        
        return seq
        
    except Exception as e:
        print(f"normalize_sequence_length 에러: {e}")
        raise

# SignLanguageModel 클래스에 메서드 추가
SignLanguageModel.normalize_sequence_length = normalize_sequence_length


### 3.3 데이터 로딩


In [7]:
def load_processed_data(self, data_path):
    """
    전처리된 landmark 데이터 로드
    
    Args:
        data_path: 데이터 경로
        
    Returns:
        X: 정규화된 시퀀스 데이터
        y: 원-핫 인코딩된 라벨
    """
    try:
        data_path = Path(data_path)
        
        # 경로 검증
        if not data_path.exists():
            raise FileNotFoundError(f"데이터 경로가 존재하지 않습니다: {data_path}")
        
        metadata_path = cf.DATA_ROOT / "dataset_metadata.csv"
        if not metadata_path.exists():
            raise FileNotFoundError(f"메타데이터 파일이 존재하지 않습니다: {metadata_path}")
        
        # 메타데이터 로드
        metadata = pd.read_csv(metadata_path)
        
        if metadata.empty:
            raise ValueError("메타데이터가 비어있습니다.")
        
        # 컬럼 검증
        required_columns = ['landmarks_file', 'word_gloss']
        missing_columns = [col for col in required_columns if col not in metadata.columns]
        if missing_columns:
            raise ValueError(f"메타데이터에 필요한 컬럼이 없습니다: {missing_columns}")
        
        # 데이터 로딩
        X, y = [], []
        failed_files = []
        
        print(f"총 {len(metadata)}개 파일 처리 중...")
        
        for idx, row in metadata.iterrows():
            try:
                landmarks_file = data_path / row['landmarks_file']
                if landmarks_file.exists():
                    landmarks = np.load(landmarks_file)
                    landmarks = self.normalize_sequence_length(landmarks)
                    X.append(landmarks)
                    y.append(row['word_gloss'])
                else:
                    failed_files.append(str(landmarks_file))
                    print(f"경고: 파일이 존재하지 않습니다: {landmarks_file}")
            except Exception as e:
                failed_files.append(str(landmarks_file))
                print(f"경고: 파일 로딩 실패 {landmarks_file}: {e}")
                continue
        
        if len(X) == 0:
            raise ValueError("로드된 데이터가 없습니다. 모든 파일 로딩에 실패했습니다.")
        
        # 데이터 검증
        sample_shape = X[0].shape
        if sample_shape[1] != self.feature_dim:
            raise ValueError(f"특성 차원 불일치: 예상={self.feature_dim}, 실제={sample_shape[1]}")
        
        if len(failed_files) > 0:
            print(f"총 {len(failed_files)}개 파일 로딩에 실패했습니다.")
        
        print(f"성공적으로 로드된 데이터: {len(X)}개")
        print(f"데이터 형태: {np.array(X).shape}")
        
        # 라벨 인코딩
        y_encoded = self.label_encoder.fit_transform(y)
        y_categorical = keras.utils.to_categorical(y_encoded, self.num_classes)
        
        return np.array(X), y_categorical
        
    except Exception as e:
        print(f"load_processed_data 에러: {e}")
        raise

# SignLanguageModel 클래스에 메서드 추가
SignLanguageModel.load_processed_data = load_processed_data


### 3.4 모델 학습


In [ ]:
def train(self, data_path, validation_split=0.2):
    """
    모델 학습
    
    Args:
        data_path: 학습 데이터 경로
        validation_split: 검증 데이터 비율
        
    Returns:
        학습 히스토리
    """
    try:
        # 입력 검증
        if not (0 < validation_split < 1):
            raise ValueError(f"validation_split은 0과 1 사이의 값이어야 합니다. 현재 값: {validation_split}")
        
        print("=== 모델 학습 시작 ===")
        
        # 데이터 로딩
        print("데이터 로딩 중...")
        X, y = self.load_processed_data(data_path)
        
        if len(X) < 10:
            raise ValueError(f"학습 데이터가 너무 적습니다. 최소 10개 이상 필요합니다. 현재: {len(X)}개")
        
        # 데이터 분할
        print("데이터 분할 중...")
        try:
            X_train, X_val, y_train, y_val = train_test_split(
                X, y, test_size=validation_split, random_state=42, stratify=y.argmax(axis=1)
            )
        except ValueError as e:
            print(f"stratify 실패, stratify 없이 분할: {e}")
            X_train, X_val, y_train, y_val = train_test_split(
                X, y, test_size=validation_split, random_state=42
            )
        
        print(f"학습 데이터: {len(X_train)}개, 검증 데이터: {len(X_val)}개")
        
        # 모델 구성
        if self.model is None:
            print("모델 구성 중...")
            self.build_model()
        else:
            print("기존 모델 사용")
        
        # 모델 저장 경로 설정
        model_save_path = cf.MODEL_SAVA_PATH
        if not model_save_path.exists():
            model_save_path.mkdir(parents=True, exist_ok=True)
            print(f"모델 저장 경로 생성: {model_save_path}")
        
        # 콜백 설정
        callbacks = [
            keras.callbacks.EarlyStopping(
                monitor='val_accuracy', 
                patience=15, 
                restore_best_weights=True, 
                verbose=1
            ),
            keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss', 
                factor=0.5, 
                patience=7, 
                min_lr=1e-7, 
                verbose=1
            ),
            keras.callbacks.ModelCheckpoint(
                model_save_path / "best_model.h5", 
                monitor='val_accuracy', 
                save_best_only=True, 
                verbose=1
            )
        ]
        
        # Mixed Precision 설정 (GPU 메모리 최적화)
        policy = keras.mixed_precision.Policy('mixed_float16')
        keras.mixed_precision.set_global_policy(policy)
        print("Mixed Precision 활성화")
        
        # 학습 실행
        print("모델 학습 시작...")
        history = self.model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=cf.EPOCHS,
            batch_size=cf.BATCH_SIZE,
            callbacks=callbacks,
            verbose=1
        )
        
        # 라벨 인코더 저장
        print("라벨 인코더 저장 중...")
        with open(model_save_path / 'label_encoder.pkl', 'wb') as f:
            pickle.dump(self.label_encoder, f)
        
        print("=== 학습 완료 ===")
        return history
        
    except Exception as e:
        print(f"train 에러: {e}")
        raise

# SignLanguageModel 클래스에 메서드 추가
SignLanguageModel.train = train


## 학습 및 검증 함수



In [ ]:
import matplotlib.pyplot as plt

def plot_history(history):
    """
    학습/검증 Accuracy & Loss 시각화
    """
    acc = history.history["accuracy"]
    val_acc = history.history["val_accuracy"]
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(acc, label="Train Acc")
    plt.plot(val_acc, label="Val Acc")
    plt.title("Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(loss, label="Train Loss")
    plt.plot(val_loss, label="Val Loss")
    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()

    plt.show()

## 학습 관련된 전처리 필요 내용 - 내일 보고 수정 필요

In [ ]:
UPPER_POSE_IDXS = [] # 상체 포즈
FACE_IDXS = [] # 손 포즈즈
# _norm_by_shoulders : 어깨 기준으로 중심점 잡는 코드. 수린이가 짠 코드로 대체하면 됨.

def extract_upperbody_hands_features(rgb_img, hands, pose, face):
    """
    상체(포즈 7점) + 좌/우 손(각 21점) + 얼굴 (25점) 사용해 (F,) 벡터 생성
    F = 7*2 + 21*2 + 21*2 + 25 * 2= 148   """
    H, W, _ = rgb_img.shape
    out = []

    # -------- Pose (상체 점만) --------
    res_pose = pose.process(rgb_img)
    pose_xy = np.zeros((33, 2), dtype=np.float32)
    has_pose = False
    if res_pose.pose_landmarks:
        has_pose = True
        for i, lm in enumerate(res_pose.pose_landmarks.landmark):
            pose_xy[i, 0] = lm.x
            pose_xy[i, 1] = lm.y

    # 어깨 기준 정규화 (어깨가 없으면 전체 0으로 둔다)
    if has_pose:
        left_sh = pose_xy[11]; right_sh = pose_xy[12]
        pose_upper = pose_xy[UPPER_POSE_IDXS]           # (7,2)
        pose_upper = _norm_by_shoulders(pose_upper, left_sh, right_sh)
    else:
        pose_upper = np.zeros((len(UPPER_POSE_IDXS), 2), dtype=np.float32)

    out.append(pose_upper.flatten())  # 7*2 = 14

    # -------- Face (subset) --------
    res_face = face.process(rgb_img)
    if res_face.multi_face_landmarks:
        face_pts = np.array(
            [(lm.x, lm.y) for i, lm in enumerate(res_face.multi_face_landmarks[0].landmark) if i in FACE_IDXS],
            dtype=np.float32
        )

        face_pts = _norm_by_shoulders(face_pts, left_sh, right_sh) if has_pose else face_pts
    else:
        face_pts = np.zeros((len(FACE_IDXS), 2), dtype=np.float32)
    out.append(face_pts.flatten())  # ex) 25 * 2 = 50

    # -------- Hands (좌/우 21점씩) --------
    res_hands = hands.process(rgb_img)

    lh = np.zeros((21, 2), dtype=np.float32)
    rh = np.zeros((21, 2), dtype=np.float32)

    if res_hands.multi_hand_landmarks and res_hands.multi_handedness:
        # 정규화 좌표 기준점: 어깨 (가능하면), 아니면 이미지 중앙
        if has_pose:
            ref_l = pose_xy[11]; ref_r = pose_xy[12]
        else:
            # 포즈가 없으면 화면 가로 1.0 기준으로 임시 스케일/원점 설정
            ref_l = np.array([0.5 - 0.1, 0.5], dtype=np.float32)
            ref_r = np.array([0.5 + 0.1, 0.5], dtype=np.float32)

        for lm, handedness in zip(res_hands.multi_hand_landmarks, res_hands.multi_handedness):
            pts = np.array([(p.x, p.y) for p in lm.landmark], dtype=np.float32)  # (21,2)
            pts = _norm_by_shoulders(pts, ref_l, ref_r)
            label = handedness.classification[0].label  # 'Left' or 'Right'
            if label.lower().startswith('left'):
                lh = pts
            else:
                rh = pts

    out.append(lh.flatten())  # 42
    out.append(rh.flatten())  # 42

    feat = np.concatenate(out, axis=0).astype(np.float32)  # (98,)
    return feat

# 사용 예시

# 얘 왜 내려가있음??

In [ ]:
# def build_model(self):
#     """
#     CNN + LSTM 하이브리드 모델 구성
    
#     구조:
#     1. TimeDistributed CNN (프레임별 특징 추출)
#     2. LSTM (시계열 패턴 학습)
#     3. Dense (분류)
#     """
#     try:
#         # 입력 검증
#         if self.sequence_length <= 0 or self.feature_dim <= 0:
#             raise ValueError(f"잘못된 입력 크기: sequence_length={self.sequence_length}, feature_dim={self.feature_dim}")
        
#         print("모델 구성 중...")
        
#         # 입력층
#         inputs = keras.Input(shape=(self.sequence_length, self.feature_dim))
        
#         # CNN을 위한 reshape
#         x = layers.Reshape((self.sequence_length, self.feature_dim, 1))(inputs)
        
#         # TimeDistributed CNN 블록 1
#         x = layers.TimeDistributed(layers.Conv1D(64, 3, activation='relu', padding='same'))(x)
#         x = layers.TimeDistributed(layers.BatchNormalization())(x)
#         x = layers.TimeDistributed(layers.MaxPooling1D(2))(x)
#         x = layers.TimeDistributed(layers.Dropout(cf.DROPOUT_RATE))(x)
        
#         # TimeDistributed CNN 블록 2
#         x = layers.TimeDistributed(layers.Conv1D(128, 3, activation='relu', padding='same'))(x)
#         x = layers.TimeDistributed(layers.BatchNormalization())(x)
#         x = layers.TimeDistributed(layers.MaxPooling1D(2))(x)
#         x = layers.TimeDistributed(layers.Dropout(cf.DROPOUT_RATE))(x)
        
#         # TimeDistributed CNN 블록 3
#         x = layers.TimeDistributed(layers.Conv1D(256, 3, activation='relu', padding='same'))(x)
#         x = layers.TimeDistributed(layers.BatchNormalization())(x)
#         x = layers.TimeDistributed(layers.GlobalMaxPooling1D())(x)
#         x = layers.TimeDistributed(layers.Dropout(cf.DROPOUT_RATE))(x)
        
#         # LSTM 블록
#         x = layers.LSTM(512, return_sequences=True, dropout=cf.DROPOUT_RATE)(x)
#         x = layers.LSTM(256, return_sequences=True, dropout=cf.DROPOUT_RATE)(x)
#         x = layers.LSTM(128, dropout=cf.DROPOUT_RATE)(x)
        
#         # 분류기
#         x = layers.Dense(512, activation='relu')(x)
#         x = layers.BatchNormalization()(x)
#         x = layers.Dropout(0.5)(x)
        
#         x = layers.Dense(256, activation='relu')(x)
#         x = layers.Dropout(0.5)(x)
        
#         outputs = layers.Dense(self.num_classes, activation='softmax')(x)
        
#         # 모델 생성
#         self.model = keras.Model(inputs, outputs)
        
#         # 컴파일
#         self.model.compile(
#             optimizer=keras.optimizers.Adam(learning_rate=cf.LEARNING_RATE, clipnorm=1.0),
#             loss='categorical_crossentropy',
#             metrics=['accuracy', 'top_5_accuracy']
#         )
        
#         print("모델 구성 완료!")
#         return self.model
        
#     except Exception as e:
#         print(f"build_model 에러: {e}")
#         raise

# # SignLanguageModel 클래스에 메서드 추가
# SignLanguageModel.build_model = build_model


NameError: name 'SignLanguageModel' is not defined